In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


/Users/aayeshaaayesha/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
#age 20-25 personal
import pandas as pd
df = pd.read_excel('person_age_20_25.xlsx')
#df = df.drop(index=0)
df

,UID,Name,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,141,Max,Elements inside Telegram,Telegram Buttons,Website Handling and UI,3/10,Yes also the reloads were really annoying.,It did not really influence my decisions.,No it nearly never matched my ranking but some...,The recommendations only met some articles and...
1,144,Antal,It was pretty hard to normally use the bot due...,The interface was not intuitive and therefore ...,"Chat system in telegram, sending vote which pr...",Using the bot provided actually even more work...,"The filtering was bad, reload of the page duri...",There was some help for aggregating the votes ...,To some extent however I still highly used my ...,Sometimes it was correct but for the most part...
2,140,Christoph,I don't know if it is that easy to use. There ...,The result of the voting was relatively clear.,I don't think it is necessary to see the votin...,Not really sure what this question refers to. ...,"Yes, errors when sending the vote. Not smooth ...",I usually shop on my own and have a definitive...,As mentioned above I usually don't need fashio...,"Overall, the recommendations were ok. The rank..."
3,142,Luka,i couldn't find to work the bot,i couldn't handle the bot too well quite compl...,without the youtube video the bot wouldnt make...,complicated process sadly,the page is reloading often and i didnt know i...,"not a lot, because i dont know who voted what ...",not a lot...why should random people rank my o...,"pretty unclear, the bot hasnt made sense for m..."
4,84,Enxhi,Getting informed when friends vote,Showing results in the end,Sharing,It is efficient because it notifies you direct...,There was no lack of smoothness,It helps me understand my friends taste toward...,"Yes, when I am in between two clothes, it is n...",The recommendations were mostly accurate
5,97,Maximilian Scheiblauer,"Wishlist, Filtering",Filtering,Style filtering with different numbers,"Alright, but still much room for improvement","Yes, removing items always causes breaks and s...",largely unuseful,Mostly not,"sometimes helpful, but most times bad"
6,100,Kasra,give feedback button,screenshot of the clothes,the sharing button,it was not very efficient. too many steps invo...,it was smooth,medium levels of help,yes it helps,it was very similar
7,96,Chacha Raphael,The messages,The ease of use,The many messages,Was efficient,Only when clicking vote,Was ok,Yes he was.,"Often was spot on, sometimes different. Genera..."
8,99,Daniel,The telegram links and messages were easy to use.,The integration with used social platforms suc...,The ranking experience sometimes was a little ...,"The recommendations were good, but the effort ...","Telegram was smooth, but mavbe there were a lo...",Sometimes it made me reconsider mv personal pr...,"For the cases where I was unsure of hesitant, ...","I perceived it as a secondary help, but my own..."
9,98,Taranpreet Kaur,the adding to the wish list,the layout,"it was not easy to go to the homepage again, t...","it could have been better, like clearing the w...",yes,"oh yhea it was helpful, because if I was confu...","yes it was useful, the bot gave me at end the ...","it was mostly slighty differnts, somtimes very..."


In [5]:
# age_20_25_personal
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]
for i in range (2, len(df.columns)):
    x = df.columns[i]
    print(sentiForCol(x))



[0.3926136, 0.11394512, 0.49344128]
[0.4338332, 0.10210642, 0.46406037]
[0.06719161, 0.3988244, 0.533984]
[0.41029027, 0.22865, 0.36105973]
[0.22094746, 0.3312609, 0.44779164]
[0.39763483, 0.17321768, 0.4291475]
[0.35371664, 0.1584651, 0.48781827]
[0.34561792, 0.16040319, 0.4939789]


In [6]:
# age_20_25_personal
#personal 26-30
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.2978794574737549
 interface negative sentiment is 0.20495864748954773
 interface neutral sentiment is 0.4971618950366974
 function positive sentiment is 0.3156188726425171
 function negative sentiment is 0.2799554467201233
 function neutral sentiment is 0.4044256806373596
 accomplish positive sentiment is 0.36565646529197693
 accomplish negative sentiment is 0.16402865946292877
 accomplish neutral sentiment is 0.4703148901462555


In [44]:
#personal 26-30
df = pd.read_excel('person_age_26_30.xlsx')
#df = df.drop(df.columns[0], axis=1)
df

,UID,Name,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,83,Mustafa,The existing of different categories and also ...,The existing of different items but I think ne...,I don't know,Was good,When remove items from wishlist I need to remo...,Was kind of helpful I reconsidered someone cho...,Honestly not that much,Almost the same
1,153,Nils,"Telegramm Start Button, That it was in Telegramm","Sorry I cannot find a good answer to this, wha...",It would be nice to have more commands. For ex...,"Hard to comment on that, but regarding the eff...","It was smooth, but I guess this also has to do...","Good help, feedback is always nice!",Since it wasn't a real shopping trip it is har...,My prefereces or pick varie from the ones the ...
2,126,Mirjana,The option to share to a social media app,getting different insights and suggestions on ...,The option where you change the intent is not ...,it was okay,"A bit yes, and in some cases I couldn't give f...","It was okay, in some cases it added up with my...",I think that in the future would be of more he...,They were close.
3,81,Dennis,Nothing! I'm even not sure if I sent my feedba...,Nothing! I wouldn't use it.,I'm even not sure if I sent my feedback to my ...,not remarkable!,yes,Let's say I got feedback.That's all.,No,"sometimes the same, sometimes not the same"
4,82,Alexander,Drag and drop,Very easy to understand interface. Absolutely ...,Once you do multiple feedback rounds in parall...,Its quite efficient. No complaints,On the phone the link on the send button didn'...,It was nice to get a second opinion. But I thi...,Yes it was helpful,It aligned pretty well In my opinion
5,130,Felix Wilberg,All Buttons,Pictures,"When I receive feedback from someone, it is no...","I like that it accumulates the opinions, so it...",Sometimes images are loading very slow.,Not really as I usually choose by my own taste...,"Not really, as I used the bot to get a sense o...",I liked it.
6,128,Laura Maria,The buttons and the text enable an intuitive i...,The big pictures of the items,"In general, it was not so easy to find the fee...",Good,No,I received helpful feedback from every team me...,Yes,"At first, I was surprised at how well the reco..."
7,128,Laura Maria,The intuitive buttons,Big pictures of the items,I don't know,Good,No,"Every team member voted, so a lot",Yes,I was surprised by how well the recommendation...
8,122,Christopher,The Telegram integration is very useful to ask...,I cannot imagine myself using the bot in real ...,No,"I cannot comment on that, efficiency regarding...",If this also concerns the online shopping expe...,"This is the most useful feature, I could imagi...","I have to say no, a friends opinion is much mo...",Very close to my own preferences.
9,124,Amila,Everything was very intuitive and therefore ea...,It is good that there is a possibility to filt...,I did not find anything unclear or difficult t...,"I think it was quite efficient, more than the ...","No, it was quite smooth. I experienced only on...","It was quite helpful, but still when shopping ...",I felt that it was a bit helpful.,It was pretty good.


In [45]:
#personal 26-30
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [46]:
#personal 26-30
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.2572651505470276
 interface negative sentiment is 0.21366386115550995
 interface neutral sentiment is 0.5290709733963013
 function positive sentiment is 0.42622852325439453
 function negative sentiment is 0.24382619559764862
 function neutral sentiment is 0.32994529604911804
 accomplish positive sentiment is 0.4643738269805908
 accomplish negative sentiment is 0.1142667755484581
 accomplish neutral sentiment is 0.4213593900203705


In [47]:
# personal_male
df = pd.read_excel('person_male.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,141,Max,Elements inside Telegram,Telegram Buttons,Website Handling and UI,3/10,Yes also the reloads were really annoying.,It did not really influence my decisions.,No it nearly never matched my ranking but some...,The recommendations only met some articles and...
1,144,Antal,It was pretty hard to normally use the bot due...,The interface was not intuitive and therefore ...,"Chat system in telegram, sending vote which pr...",Using the bot provided actually even more work...,"The filtering was bad, reload of the page duri...",There was some help for aggregating the votes ...,To some extent however I still highly used my ...,Sometimes it was correct but for the most part...
2,140,Christoph,I don't know if it is that easy to use. There ...,The result of the voting was relatively clear.,I don't think it is necessary to see the votin...,Not really sure what this question refers to. ...,"Yes, errors when sending the vote. Not smooth ...",I usually shop on my own and have a definitive...,As mentioned above I usually don't need fashio...,"Overall, the recommendations were ok. The rank..."
3,142,Luka,i couldn't find to work the bot,i couldn't handle the bot too well quite compl...,without the youtube video the bot wouldnt make...,complicated process sadly,the page is reloading often and i didnt know i...,"not a lot, because i dont know who voted what ...",not a lot...why should random people rank my o...,"pretty unclear, the bot hasnt made sense for m..."
4,83,Mustafa,The existing of different categories and also ...,The existing of different items but I think ne...,I don't know,Was good,When remove items from wishlist I need to remo...,Was kind of helpful I reconsidered someone cho...,Honestly not that much,Almost the same
5,153,Nils,"Telegramm Start Button, That it was in Telegramm","Sorry I cannot find a good answer to this, wha...",It would be nice to have more commands. For ex...,"Hard to comment on that, but regarding the eff...","It was smooth, but I guess this also has to do...","Good help, feedback is always nice!",Since it wasn't a real shopping trip it is har...,My prefereces or pick varie from the ones the ...
6,97,Maximilian Scheiblauer,"Wishlist, Filtering",Filtering,Style filtering with different numbers,"Alright, but still much room for improvement","Yes, removing items always causes breaks and s...",largely unuseful,Mostly not,"sometimes helpful, but most times bad"
7,100,Kasra,give feedback button,screenshot of the clothes,the sharing button,it was not very efficient. too many steps invo...,it was smooth,medium levels of help,yes it helps,it was very similar
8,96,Chacha Raphael,The messages,The ease of use,The many messages,Was efficient,Only when clicking vote,Was ok,Yes he was.,"Often was spot on, sometimes different. Genera..."
9,99,Daniel,The telegram links and messages were easy to use.,The integration with used social platforms suc...,The ranking experience sometimes was a little ...,"The recommendations were good, but the effort ...","Telegram was smooth, but mavbe there were a lo...",Sometimes it made me reconsider mv personal pr...,"For the cases where I was unsure of hesitant, ...","I perceived it as a secondary help, but my own..."


In [48]:
# personal_male
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [49]:
# personal_male
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.27043116092681885
 interface negative sentiment is 0.22017067670822144
 interface neutral sentiment is 0.5093981623649597
 function positive sentiment is 0.34601205587387085
 function negative sentiment is 0.2713828682899475
 function neutral sentiment is 0.38260507583618164
 accomplish positive sentiment is 0.3545960485935211
 accomplish negative sentiment is 0.1579098105430603
 accomplish neutral sentiment is 0.4874941408634186


In [14]:
# female personal
df = pd.read_excel('person_female.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,119,Fung Yee,the ranking is easy to apply,the bot has pictures of the clothes you could ...,"you need to press ""start"" button and share to ...",low efficiency,no,actually not. i found out the recommendations ...,no. the recommendations from their feedback is...,the ranking result of mine and theirs is not c...
1,84,Enxhi,Getting informed when friends vote,Showing results in the end,Sharing,It is efficient because it notifies you direct...,There was no lack of smoothness,It helps me understand my friends taste toward...,"Yes, when I am in between two clothes, it is n...",The recommendations were mostly accurate
2,98,Taranpreet Kaur,the adding to the wish list,the layout,"it was not easy to go to the homepage again, t...","it could have been better, like clearing the w...",yes,"oh yhea it was helpful, because if I was confu...","yes it was useful, the bot gave me at end the ...","it was mostly slighty differnts, somtimes very..."
3,126,Mirjana,The option to share to a social media app,getting different insights and suggestions on ...,The option where you change the intent is not ...,it was okay,"A bit yes, and in some cases I couldn't give f...","It was okay, in some cases it added up with my...",I think that in the future would be of more he...,They were close.
4,128,Laura Maria,The buttons and the text enable an intuitive i...,The big pictures of the items,"In general, it was not so easy to find the fee...",Good,No,I received helpful feedback from every team me...,Yes,"At first, I was surprised at how well the reco..."
5,128,Laura Maria,The intuitive buttons,Big pictures of the items,-,Good,No,"Every team member voted, so a lot",Yes,I was surprised by how well the recommendation...
6,102,Theresa,"The buttons make it very easy to use, also tha...","that its usable for telegram, easy to send to ...",you dont know how many of your friends have al...,ok,"every picture had to be clicked separately, a ...","not that much, i usually dont ask often about ...",can be nice to see someone has the same taste ...,"sometimes we had the same preference, sometime..."
7,104,Nadia,it was easy to ask for and get feedback from m...,very easy to share the wishlist with my friends,no explicit info on the voting progress of my ...,low-okay,"no easy way to navigate between pictures, I ha...",their votes made me consider/be sure of my cho...,"yes, I got good recommendations from the group...",the votings would encourage me to buy the clothes
8,103,Marina,"^ in voting section and drag & drop, just clic...",sharing of wishlist was very easy and receivin...,"no use of 'unsubscribe' button, very unclear h...",very low,"no good inteface fpr voting: images too small,...",not very helpful,i would say no - because i have already select...,i dont understand the question.. the bot only ...
9,124,Amila,Everything was very intuitive and therefore ea...,It is good that there is a possibility to filt...,I did not find anything unclear or difficult t...,"I think it was quite efficient, more than the ...","No, it was quite smooth. I experienced only on...","It was quite helpful, but still when shopping ...",I felt that it was a bit helpful.,It was pretty good.


In [15]:
# female personal
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]
for i in range (2, len(df.columns)):
    x = df.columns[i]
    print(sentiForCol(x))

[0.39855036, 0.09407319, 0.50737643]
[0.44812158, 0.09516219, 0.4567162]
[0.087396264, 0.41322282, 0.49938092]
[0.4280273, 0.15975647, 0.41221625]
[0.24603651, 0.31310725, 0.44085625]
[0.49505767, 0.16990411, 0.33503821]
[0.47733644, 0.16548143, 0.35718215]
[0.3646394, 0.17546734, 0.45989326]


In [16]:
# female personal
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])

print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.3113560676574707
 interface negative sentiment is 0.20081940293312073
 interface neutral sentiment is 0.48782452940940857
 function positive sentiment is 0.33703190088272095
 function negative sentiment is 0.23643186688423157
 function neutral sentiment is 0.4265362620353699
 accomplish positive sentiment is 0.44567784667015076
 accomplish negative sentiment is 0.17028430104255676
 accomplish neutral sentiment is 0.38403788208961487


In [17]:
# personal_arts
df = pd.read_excel('person_arts.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,141,Max,Elements inside Telegram,Telegram Buttons,Website Handling and UI,3/10,Yes also the reloads were really annoying.,It did not really influence my decisions.,No it nearly never matched my ranking but some...,The recommendations only met some articles and...
1,84,Enxhi,Getting informed when friends vote,Showing results in the end,Sharing,It is efficient because it notifies you direct...,There was no lack of smoothness,It helps me understand my friends taste toward...,"Yes, when I am in between two clothes, it is n...",The recommendations were mostly accurate
2,97,Maximilian Scheiblauer,"Wishlist, Filtering",Filtering,Style filtering with different numbers,"Alright, but still much room for improvement","Yes, removing items always causes breaks and s...",largely unuseful,Mostly not,"sometimes helpful, but most times bad"
3,99,Daniel,The telegram links and messages were easy to use.,The integration with used social platforms suc...,The ranking experience sometimes was a little ...,"The recommendations were good, but the effort ...","Telegram was smooth, but mavbe there were a lo...",Sometimes it made me reconsider mv personal pr...,"For the cases where I was unsure of hesitant, ...","I perceived it as a secondary help, but my own..."
4,98,Taranpreet Kaur,the adding to the wish list,the layout,"it was not easy to go to the homepage again, t...","it could have been better, like clearing the w...",yes,"oh yhea it was helpful, because if I was confu...","yes it was useful, the bot gave me at end the ...","it was mostly slighty differnts, somtimes very..."
5,85,Emirhan,The link which drives directly to the Telegram...,Sharing the list which people liked in a reall...,Gruoping the list or maybe can be added by the...,It is really efficent to share list that peopl...,I haven't came across any.,Most of them is releated to my taste so those ...,It can assist to reach a decision by geting fr...,It was close to my preferences.
6,127,Benjámin,category and intent selection interfaces are c...,I actually like the heart symbol indicates the...,style selection segment options are just numbe...,The sharing feature is very nice and useful to...,Everything was ok.,It is a good tool to get fast and useful feedb...,"yes, definitely. It is an easy to understand b...",The similarity was quite high. I really enjoye...
7,130,Felix Wilberg,All Buttons,Pictures,"When I receive feedback from someone, it is no...","I like that it accumulates the opinions, so it...",Sometimes images are loading very slow.,Not really as I usually choose by my own taste...,"Not really, as I used the bot to get a sense o...",I liked it.
8,128,Laura Maria,The buttons and the text enable an intuitive i...,The big pictures of the items,"In general, it was not so easy to find the fee...",Good,No,I received helpful feedback from every team me...,Yes,"At first, I was surprised at how well the reco..."
9,128,Laura Maria,The intuitive buttons,Big pictures of the items,-,Good,No,"Every team member voted, so a lot",Yes,I was surprised by how well the recommendation...


In [18]:
# personal_arts
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [19]:
# personal_arts
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.31517493724823
 interface negative sentiment is 0.15819667279720306
 interface neutral sentiment is 0.5266283750534058
 function positive sentiment is 0.3580801486968994
 function negative sentiment is 0.2281385064125061
 function neutral sentiment is 0.4137813448905945
 accomplish positive sentiment is 0.43076154589653015
 accomplish negative sentiment is 0.11080441623926163
 accomplish neutral sentiment is 0.4584340453147888


In [20]:
#personal_entertainment
df = pd.read_excel('person_entertain.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,119,Fung Yee,the ranking is easy to apply,the bot has pictures of the clothes you could ...,"you need to press ""start"" button and share to ...",low efficiency,no,actually not. i found out the recommendations ...,no. the recommendations from their feedback is...,the ranking result of mine and theirs is not c...
1,141,Max,Elements inside Telegram,Telegram Buttons,Website Handling and UI,3/10,Yes also the reloads were really annoying.,It did not really influence my decisions.,No it nearly never matched my ranking but some...,The recommendations only met some articles and...
2,140,Christoph,I don't know if it is that easy to use. There ...,The result of the voting was relatively clear.,I don't think it is necessary to see the votin...,Not really sure what this question refers to. ...,"Yes, errors when sending the vote. Not smooth ...",I usually shop on my own and have a definitive...,As mentioned above I usually don't need fashio...,"Overall, the recommendations were ok. The rank..."
3,142,Luka,i couldn't find to work the bot,i couldn't handle the bot too well quite compl...,without the youtube video the bot wouldnt make...,complicated process sadly,the page is reloading often and i didnt know i...,"not a lot, because i dont know who voted what ...",not a lot...why should random people rank my o...,"pretty unclear, the bot hasnt made sense for m..."
4,84,Enxhi,Getting informed when friends vote,Showing results in the end,Sharing,It is efficient because it notifies you direct...,There was no lack of smoothness,It helps me understand my friends taste toward...,"Yes, when I am in between two clothes, it is n...",The recommendations were mostly accurate
5,153,Nils,"Telegramm Start Button, That it was in Telegramm","Sorry I cannot find a good answer to this, wha...",It would be nice to have more commands. For ex...,"Hard to comment on that, but regarding the eff...","It was smooth, but I guess this also has to do...","Good help, feedback is always nice!",Since it wasn't a real shopping trip it is har...,My prefereces or pick varie from the ones the ...
6,96,Chacha Raphael,The messages,The ease of use,The many messages,Was efficient,Only when clicking vote,Was ok,Yes he was.,"Often was spot on, sometimes different. Genera..."
7,98,Taranpreet Kaur,the adding to the wish list,the layout,"it was not easy to go to the homepage again, t...","it could have been better, like clearing the w...",yes,"oh yhea it was helpful, because if I was confu...","yes it was useful, the bot gave me at end the ...","it was mostly slighty differnts, somtimes very..."
8,82,Alexander,Drag and drop,Very easy to understand interface. Absolutely ...,Once you do multiple feedback rounds in parall...,Its quite efficient. No complaints,On the phone the link on the send button didn'...,It was nice to get a second opinion. But I thi...,Yes it was helpful,It aligned pretty well In my opinion
9,85,Emirhan,The link which drives directly to the Telegram...,Sharing the list which people liked in a reall...,Gruoping the list or maybe can be added by the...,It is really efficent to share list that peopl...,I haven't came across any.,Most of them is releated to my taste so those ...,It can assist to reach a decision by geting fr...,It was close to my preferences.


In [21]:
#personal_entertain
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [22]:
#personal_entertain
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.2941417694091797
 interface negative sentiment is 0.2257571816444397
 interface neutral sentiment is 0.4801010489463806
 function positive sentiment is 0.27784544229507446
 function negative sentiment is 0.2913314700126648
 function neutral sentiment is 0.43082308769226074
 accomplish positive sentiment is 0.3866373300552368
 accomplish negative sentiment is 0.19572670757770538
 accomplish neutral sentiment is 0.417635977268219


In [23]:
#personal_sports
df = pd.read_excel('person_sports.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,119,Fung Yee,the ranking is easy to apply,the bot has pictures of the clothes you could ...,"you need to press ""start"" button and share to ...",low efficiency,no,actually not. i found out the recommendations ...,no. the recommendations from their feedback is...,the ranking result of mine and theirs is not c...
1,141,Max,Elements inside Telegram,Telegram Buttons,Website Handling and UI,3/10,Yes also the reloads were really annoying.,It did not really influence my decisions.,No it nearly never matched my ranking but some...,The recommendations only met some articles and...
2,144,Antal,It was pretty hard to normally use the bot due...,The interface was not intuitive and therefore ...,"Chat system in telegram, sending vote which pr...",Using the bot provided actually even more work...,"The filtering was bad, reload of the page duri...",There was some help for aggregating the votes ...,To some extent however I still highly used my ...,Sometimes it was correct but for the most part...
3,140,Christoph,I don't know if it is that easy to use. There ...,The result of the voting was relatively clear.,I don't think it is necessary to see the votin...,Not really sure what this question refers to. ...,"Yes, errors when sending the vote. Not smooth ...",I usually shop on my own and have a definitive...,As mentioned above I usually don't need fashio...,"Overall, the recommendations were ok. The rank..."
4,142,Luka,i couldn't find to work the bot,i couldn't handle the bot too well quite compl...,without the youtube video the bot wouldnt make...,complicated process sadly,the page is reloading often and i didnt know i...,"not a lot, because i dont know who voted what ...",not a lot...why should random people rank my o...,"pretty unclear, the bot hasnt made sense for m..."
...,...,...,...,...,...,...,...,...,...,...
57,135,Yannik,Instructions were clear.\n,Overview of shopping cart.,Process of giving recommendation had to be don...,Okay.,"Yes, the telegram interface makes it a little ...",Reasonable amount. The reasons behind the rank...,A little. But without reasoning not really.,Mostly accurate.
58,137,Roman,"It was clear, what to expect from each step",Generated pictures are easy to comprehend also...,"It was a bit unclear, whether feedback is prov...","It was quite efficient, no technical issues",It was fine,Suggestions aligned with something I would lik...,Not entirely as I would say I have other facto...,It was fine
59,138,Kristian,.,.,.,.,.,.,.,.
60,139,Rebeka,It is easy to ask friends for recommendations ...,The way items can be sorted when giving recomm...,It's hard to keep track of my wishlists and al...,It could be more efficient with a better navig...,"Yes, the navigation is not smooth. I click on ...","I got all the help I could possibly get, as al...",I wouldn't say the bot helped me much. My frie...,"Out of 5, only one recommendation was complete..."


In [24]:
#personal_sports
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [25]:
#personal_sports
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.27437156438827515
 interface negative sentiment is 0.22278721630573273
 interface neutral sentiment is 0.5028412342071533
 function positive sentiment is 0.33474642038345337
 function negative sentiment is 0.2738184928894043
 function neutral sentiment is 0.39143508672714233
 accomplish positive sentiment is 0.36503899097442627
 accomplish negative sentiment is 0.16803911328315735
 accomplish neutral sentiment is 0.4669218957424164


In [26]:
#personal_study
df = pd.read_excel('person_study.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,144,Antal,It was pretty hard to normally use the bot due...,The interface was not intuitive and therefore ...,"Chat system in telegram, sending vote which pr...",Using the bot provided actually even more work...,"The filtering was bad, reload of the page duri...",There was some help for aggregating the votes ...,To some extent however I still highly used my ...,Sometimes it was correct but for the most part...
1,140,Christoph,I don't know if it is that easy to use. There ...,The result of the voting was relatively clear.,I don't think it is necessary to see the votin...,Not really sure what this question refers to. ...,"Yes, errors when sending the vote. Not smooth ...",I usually shop on my own and have a definitive...,As mentioned above I usually don't need fashio...,"Overall, the recommendations were ok. The rank..."
2,83,Mustafa,The existing of different categories and also ...,The existing of different items but I think ne...,I don't know.,Was good,When remove items from wishlist I need to remo...,Was kind of helpful I reconsidered someone cho...,Honestly not that much,Almost the same
3,153,Nils,"Telegramm Start Button, That it was in Telegramm","Sorry I cannot find a good answer to this, wha...",It would be nice to have more commands. For ex...,"Hard to comment on that, but regarding the eff...","It was smooth, but I guess this also has to do...","Good help, feedback is always nice!",Since it wasn't a real shopping trip it is har...,My prefereces or pick varie from the ones the ...
4,96,Chacha Raphael,The messages,The ease of use,The many messages,Was efficient,Only when clicking vote,Was ok,Yes he was.,"Often was spot on, sometimes different. Genera..."
5,130,Felix Wilberg,All Buttons,Pictures,"When I receive feedback from someone, it is no...","I like that it accumulates the opinions, so it...",Sometimes images are loading very slow.,Not really as I usually choose by my own taste...,"Not really, as I used the bot to get a sense o...",I liked it.
6,101,Jörg,one-click-solution (while ranking),I used it on my mobile - it was quite small. I...,"It's okay after some time, but you have to get...",okay.,Yes. Depending on the browser I wasn't able to...,I found especially helpful when others had ver...,Only in the two extreme cases: everyone encour...,very different
7,105,Philipp,Giving feedback was easy,"To use the bot with telegram, so everything wa...",subscribe button,"it was ok, bot was easy to use, just press the...",no,It could maybe help in confirming wether I mad...,"Not per se, but theoretically could be useful ...",they were mixed
8,104,Nadia,it was easy to ask for and get feedback from m...,very easy to share the wishlist with my friends,no explicit info on the voting progress of my ...,low-okay,"no easy way to navigate between pictures, I ha...",their votes made me consider/be sure of my cho...,"yes, I got good recommendations from the group...",the votings would encourage me to buy the clothes
9,103,Marina,"^ in voting section and drag & drop, just clic...",sharing of wishlist was very easy and receivin...,"no use of 'unsubscribe' button, very unclear h...",very low,"no good inteface fpr voting: images too small,...",not very helpful,i would say no - because i have already select...,i dont understand the question.. the bot only ...


In [27]:
#personal_study
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [28]:
#personal_study
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.291564404964447
 interface negative sentiment is 0.242141455411911
 interface neutral sentiment is 0.46629413962364197
 function positive sentiment is 0.34107905626296997
 function negative sentiment is 0.2809789180755615
 function neutral sentiment is 0.3779420554637909
 accomplish positive sentiment is 0.3775542080402374
 accomplish negative sentiment is 0.13902483880519867
 accomplish neutral sentiment is 0.4834209382534027


In [29]:
#personal_nature
df = pd.read_excel('person_nature.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,140,Christoph,I don't know if it is that easy to use. There ...,The result of the voting was relatively clear.,I don't think it is necessary to see the votin...,Not really sure what this question refers to. ...,"Yes, errors when sending the vote. Not smooth ...",I usually shop on my own and have a definitive...,As mentioned above I usually don't need fashio...,"Overall, the recommendations were ok. The rank..."
1,142,Luka,i couldn't find to work the bot,i couldn't handle the bot too well quite compl...,without the youtube video the bot wouldnt make...,complicated process sadly,the page is reloading often and i didnt know i...,"not a lot, because i dont know who voted what ...",not a lot...why should random people rank my o...,"pretty unclear, the bot hasnt made sense for m..."
2,83,Mustafa,The existing of different categories and also ...,The existing of different items but I think ne...,I don't know,Was good,When remove items from wishlist I need to remo...,Was kind of helpful I reconsidered someone cho...,Honestly not that much,Almost the same
3,84,Enxhi,Getting informed when friends vote,Showing results in the end,Sharing,It is efficient because it notifies you direct...,There was no lack of smoothness,It helps me understand my friends taste toward...,"Yes, when I am in between two clothes, it is n...",The recommendations were mostly accurate
4,100,Kasra,give feedback button,screenshot of the clothes,the sharing button,it was not very efficient. too many steps invo...,it was smooth,medium levels of help,yes it helps,it was very similar
5,96,Chacha Raphael,The messages,The ease of use,The many messages,Was efficient,Only when clicking vote,Was ok,Yes he was.,"Often was spot on, sometimes different. Genera..."
6,98,Taranpreet Kaur,the adding to the wish list,the layout,"it was not easy to go to the homepage again, t...","it could have been better, like clearing the w...",yes,"oh yhea it was helpful, because if I was confu...","yes it was useful, the bot gave me at end the ...","it was mostly slighty differnts, somtimes very..."
7,126,Mirjana,The option to share to a social media app,getting different insights and suggestions on ...,The option where you change the intent is not ...,it was okay,"A bit yes, and in some cases I couldn't give f...","It was okay, in some cases it added up with my...",I think that in the future would be of more he...,They were close.
8,81,Dennis,Nothing! I'm even not sure if I sent my feedba...,Nothing! I wouldn't use it.,I'm even not sure if I sent my feedback to my ...,not remarkable!,yes,Let's say I got feedback.That's all.,No,"sometimes the same, sometimes not the same"
9,85,Emirhan,The link which drives directly to the Telegram...,Sharing the list which people liked in a reall...,Gruoping the list or maybe can be added by the...,It is really efficent to share list that peopl...,I haven't came across any.,Most of them is releated to my taste so those ...,It can assist to reach a decision by geting fr...,It was close to my preferences.


In [30]:
#personal_nature
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [31]:
#personal_nature
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])

print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.2978765368461609
 interface negative sentiment is 0.228193998336792
 interface neutral sentiment is 0.4739294648170471
 function positive sentiment is 0.36595362424850464
 function negative sentiment is 0.26446810364723206
 function neutral sentiment is 0.3695782423019409
 accomplish positive sentiment is 0.4073113203048706
 accomplish negative sentiment is 0.14606249332427979
 accomplish neutral sentiment is 0.4466261565685272


In [32]:
#personal low-experience
df = pd.read_excel('person_lowLevel.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,83,Mustafa,The existing of different categories and also ...,The existing of different items but I think ne...,I don't know,Was good,When remove items from wishlist I need to remo...,Was kind of helpful I reconsidered someone cho...,Honestly not that much,Almost the same
1,85,Emirhan,The link which drives directly to the Telegram...,Sharing the list which people liked in a reall...,Gruoping the list or maybe can be added by the...,It is really efficent to share list that peopl...,I haven't came across any.,Most of them is releated to my taste so those ...,It can assist to reach a decision by geting fr...,It was close to my preferences.
2,102,Theresa,"The buttons make it very easy to use, also tha...","that its usable for telegram, easy to send to ...",you dont know how many of your friends have al...,ok,"every picture had to be clicked separately, a ...","not that much, i usually dont ask often about ...",can be nice to see someone has the same taste ...,"sometimes we had the same preference, sometime..."
3,105,Philipp,Giving feedback was easy,"To use the bot with telegram, so everything wa...",subscribe button,"it was ok, bot was easy to use, just press the...",no,It could maybe help in confirming wether I mad...,"Not per se, but theoretically could be useful ...",they were mixed
4,103,Marina,"^ in voting section and drag & drop, just clic...",sharing of wishlist was very easy and receivin...,"no use of 'unsubscribe' button, very unclear h...",very low,"no good inteface fpr voting: images too small,...",not very helpful,i would say no - because i have already select...,i dont understand the question.. the bot only ...
5,145,Sebastian,"ranking, overall process was quite simple and ...",simplicity only few button clicks neccessary,"replies from each friend with the ""show result...","very efficient, simple and straight forward",no,some recommendations helped when I was unsure ...,sometimes (~1 out of 5),mostly a bit different but not totally different
6,125,Riad,Whole Bot interface is very intuitive,The easy sharing of selections,"At times, it seems that too many steps are nee...",The efficiency level is satisfactory,There were no noticeable lags while using the bot,The feedback from my friends was very useful,Sometimes the diversity of clothing items left...,The recommendations where matching to a good e...
7,132,Johannes Hufnagl,The recommendation was easy to use with the ar...,The design of the images,The voting system can be closed I didn't under...,Very efficient and fast to use,No this was ok,Not much I don't care for other people's opini...,It was nice to see if other people would also ...,SOmetimes well sometimes not so much
8,88,Elias Wolfgang,"The ""Give Feedback"" button.",The visual ranking of the items.,"The ""unsubscribe"" button.","Okay, but sometimes it took a bit longer to load.","Yes, sometimes.","A lot, since otherwise I would have to send th...","Yes, it was!",Fairly good.
9,138,Kristian,.,.,.,.,.,.,.,.


In [33]:
#personal low-experience
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [34]:
#personal low-experience
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])
print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.35230791568756104
 interface negative sentiment is 0.12576694786548615
 interface neutral sentiment is 0.521925151348114
 function positive sentiment is 0.3724517524242401
 function negative sentiment is 0.19117340445518494
 function neutral sentiment is 0.43637484312057495
 accomplish positive sentiment is 0.332291841506958
 accomplish negative sentiment is 0.13955703377723694
 accomplish neutral sentiment is 0.5281511545181274


In [35]:
#personal middle-experience
df = pd.read_excel('person_medLevel.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,119,Fung Yee,the ranking is easy to apply,the bot has pictures of the clothes you could ...,"you need to press ""start"" button and share to ...",low efficiency,no,actually not. i found out the recommendations ...,no. the recommendations from their feedback is...,the ranking result of mine and theirs is not c...
1,141,Max,Elements inside Telegram,Telegram Buttons,Website Handling and UI,3/10,Yes also the reloads were really annoying.,It did not really influence my decisions.,No it nearly never matched my ranking but some...,The recommendations only met some articles and...
2,140,Christoph,I don't know if it is that easy to use. There ...,The result of the voting was relatively clear.,I don't think it is necessary to see the votin...,Not really sure what this question refers to. ...,"Yes, errors when sending the vote. Not smooth ...",I usually shop on my own and have a definitive...,As mentioned above I usually don't need fashio...,"Overall, the recommendations were ok. The rank..."
3,142,Luka,i couldn't find to work the bot,i couldn't handle the bot too well quite compl...,without the youtube video the bot wouldnt make...,complicated process sadly,the page is reloading often and i didnt know i...,"not a lot, because i dont know who voted what ...",not a lot...why should random people rank my o...,"pretty unclear, the bot hasnt made sense for m..."
4,153,Nils,"Telegramm Start Button, That it was in Telegramm","Sorry I cannot find a good answer to this, wha...",It would be nice to have more commands. For ex...,"Hard to comment on that, but regarding the eff...","It was smooth, but I guess this also has to do...","Good help, feedback is always nice!",Since it wasn't a real shopping trip it is har...,My prefereces or pick varie from the ones the ...
5,97,Maximilian Scheiblauer,"Wishlist, Filtering",Filtering,Style filtering with different numbers,"Alright, but still much room for improvement","Yes, removing items always causes breaks and s...",largely unuseful,Mostly not,"sometimes helpful, but most times bad"
6,100,Kasra,give feedback button,screenshot of the clothes,the sharing button,it was not very efficient. too many steps invo...,it was smooth,medium levels of help,yes it helps,it was very similar
7,98,Taranpreet Kaur,the adding to the wish list,the layout,"it was not easy to go to the homepage again, t...","it could have been better, like clearing the w...",yes,"oh yhea it was helpful, because if I was confu...","yes it was useful, the bot gave me at end the ...","it was mostly slighty differnts, somtimes very..."
8,81,Dennis,Nothing! I'm even not sure if I sent my feedba...,Nothing! I wouldn't use it.,I'm even not sure if I sent my feedback to my ...,not remarkable!,yes,Let's say I got feedback.That's all.,No,"sometimes the same, sometimes not the same"
9,82,Alexander,Drag and drop,Very easy to understand interface. Absolutely ...,Once you do multiple feedback rounds in parall...,Its quite efficient. No complaints,On the phone the link on the send button didn'...,It was nice to get a second opinion. But I thi...,Yes it was helpful,It aligned pretty well In my opinion


In [36]:
#personal middle-experience
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [37]:
#personal middle-experience
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])

print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.24975956976413727
 interface negative sentiment is 0.23270554840564728
 interface neutral sentiment is 0.5175348520278931
 function positive sentiment is 0.3376672863960266
 function negative sentiment is 0.2893218398094177
 function neutral sentiment is 0.37301087379455566
 accomplish positive sentiment is 0.34274038672447205
 accomplish negative sentiment is 0.1849433332681656
 accomplish neutral sentiment is 0.47231629490852356


In [38]:
#personal high-experience
df = pd.read_excel('person_goodLevel.xlsx')
df = df.drop(df.columns[0], axis=1)
df

,UID,Unnamed: 1,interface1p,interface2p,interface3p,function1p,function2p,accomp1p,accomp2p,accomp3p
0,84,Enxhi,Getting informed when friends vote,Showing results in the end,Sharing,It is efficient because it notifies you direct...,There was no lack of smoothness,It helps me understand my friends taste toward...,"Yes, when I am in between two clothes, it is n...",The recommendations were mostly accurate
1,96,Chacha Raphael,The messages,The ease of use,The many messages,Was efficient,Only when clicking vote,Was ok,Yes he was.,"Often was spot on, sometimes different. Genera..."
2,99,Daniel,The telegram links and messages were easy to use.,The integration with used social platforms suc...,The ranking experience sometimes was a little ...,"The recommendations were good, but the effort ...","Telegram was smooth, but mavbe there were a lo...",Sometimes it made me reconsider mv personal pr...,"For the cases where I was unsure of hesitant, ...","I perceived it as a secondary help, but my own..."
3,127,Benjámin,category and intent selection interfaces are c...,I actually like the heart symbol indicates the...,style selection segment options are just numbe...,The sharing feature is very nice and useful to...,Everything was ok.,It is a good tool to get fast and useful feedb...,"yes, definitely. It is an easy to understand b...",The similarity was quite high. I really enjoye...
4,124,Amila,Everything was very intuitive and therefore ea...,It is good that there is a possibility to filt...,I did not find anything unclear or difficult t...,"I think it was quite efficient, more than the ...","No, it was quite smooth. I experienced only on...","It was quite helpful, but still when shopping ...",I felt that it was a bit helpful.,It was pretty good.
5,94,Waldemar,the fact that you can easily share your wishli...,the fact that you can easily share your wishli...,"What the scenario when the bot would be used, ...","Some of the experience didn't seem refined, su...","For the sakes of this research, it wasn't alwa...",They picked out some interesting clothes for me.,I'm not sure. This is a fictitious scenario. I...,Sometimes coincided. Sometimes didn't at all.
6,155,Tobias,It helped and was a nice feature to see who vo...,To be able to have an overview of each vote an...,The number of notifications and in general the...,Sometimes the buttons in the Telegram interfac...,Sometimes the buttons in the Telegram interfac...,The friends did barely help in the feedback. B...,The bot did make the shopping experience more ...,My own wishlist did have better results than t...
7,115,Kim,"the telegram interface, since I am already fam...","intuitive UI, nice with the pictures","over all too much text, could easily loose ove...",was sufficient,-,was good,"still found the recommendations interesting, t...","most of the times, i liked the ratings, just w..."
8,114,Anastasia,It was easy to select the items and just make ...,nice pictures of the clothes and their ranking,I think it was quite unfriendly when i change ...,It is okay,Didn't like to start the bot all over again to...,"yes, I was very pleased to receive feedback fr...","yeah, sometimes it really recommended nice thi...",it is fine


In [39]:
#personal high-experience
from statistics import mean 
def sentiForCol(x):
    pos_ls = []
    neg_ls = []
    neu_ls = []
    for i in range(df.shape[0]):
        text = df.iloc[i][x]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
            elif l=='negative':
                neg_ls.append(s)
            elif l=='neutral':
                neu_ls.append(s)
    mean_pos = mean(pos_ls)
    mean_neg = mean(neg_ls)
    mean_neu = mean(neu_ls)
    return [mean_pos, mean_neg, mean_neu]

In [40]:
#personal high-experience
all_sentiment = []
for i in range (2, len(df.columns)):
    x = df.columns[i]
    y = sentiForCol(x)
    all_sentiment.append(y)

interface_pos_list = []
interface_neg_list = []
interface_neu_list = []
func_pos_list = []
func_neg_list = []
func_neu_list = []
accomp_pos_list = []
accomp_neg_list = []
accomp_neu_list = []

for j in range(0, len(all_sentiment)):
    if j <= 2:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                interface_pos_list.append(all_sentiment[j][h])
            if h == 1:
                interface_neg_list.append(all_sentiment[j][h])
            if h == 2:
                interface_neu_list.append(all_sentiment[j][h])
    if j == 3 or j == 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                func_pos_list.append(all_sentiment[j][h])
            if h == 1:
                func_neg_list.append(all_sentiment[j][h])
            if h == 2:
                func_neu_list.append(all_sentiment[j][h])
    if j > 4:
        for h in range(0, len(all_sentiment[j])):
            if h == 0:
                accomp_pos_list.append(all_sentiment[j][h])
            if h == 1:
                accomp_neg_list.append(all_sentiment[j][h])
            if h == 2:
                accomp_neu_list.append(all_sentiment[j][h])

print(f""" interface positive sentiment is {mean(interface_pos_list)}""")
print(f""" interface negative sentiment is {mean(interface_neg_list)}""")
print(f""" interface neutral sentiment is {mean(interface_neu_list)}""")
print(f""" function positive sentiment is {mean(func_pos_list)}""")
print(f""" function negative sentiment is {mean(func_neg_list)}""")
print(f""" function neutral sentiment is {mean(func_neu_list)}""")
print(f""" accomplish positive sentiment is {mean(accomp_pos_list)}""")
print(f""" accomplish negative sentiment is {mean(accomp_neg_list)}""")
print(f""" accomplish neutral sentiment is {mean(accomp_neu_list)}""")

 interface positive sentiment is 0.46796274185180664
 interface negative sentiment is 0.16795042157173157
 interface neutral sentiment is 0.3640868663787842
 function positive sentiment is 0.36885303258895874
 function negative sentiment is 0.1709265112876892
 function neutral sentiment is 0.46022045612335205
 accomplish positive sentiment is 0.6327494978904724
 accomplish negative sentiment is 0.0937524363398552
 accomplish neutral sentiment is 0.2734980583190918
